In [8]:
import sys
import numpy as np
sys.path.append(r'../')

# from spl_train import run_spl

In [9]:
import pandas as pd
import time
from score import simplify_eq
from spl_base import SplBase
from spl_task_utils import *

import numpy as np
from numpy import *
from sympy import simplify, expand
from scipy.optimize import minimize
from contextlib import contextmanager
import threading
import _thread
import time
import signal

import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

RULEMAP = ['A->(A+A)', 'A->(A-A)', 'A->(A*A)', 'A->(A/A)', 'A->x', 
           'A->(x**2)', 'A->(x**4)', 
           'A->(A*C)', 'A->exp(x)', 'A->cos(C*x)', 'A->sin(C*x)']

class TimeoutException(Exception):
    def __init__(self, msg=''):
        self.msg = msg

@contextmanager
def time_limit(seconds, msg=''):
    def _handler(signum, frame):
        raise TimeOutException("Timed out for operation {}".format(msg))
    signal.signal(signal.SIGALRM, _handler)
    signal.alarm(int(seconds))
    try:
        yield
    except TimeOutException:
        raise 
    finally:
        signal.alarm(0)

    

def simplify_eq(eq):
    return str(expand(simplify(eq)))

def simplify_eqs(eqs):
    simplified_eqs = []
    for eq in eqs:
        simplified_eqs.append(str(expand(simplify(eq))))
    return simplified_eqs


def prune_poly_c(eq):
    '''
    if polynomial of C appear in eq, reduce to C for computational efficiency. 
    '''
    eq = simplify_eq(eq)
    if 'C**' in eq:
        c_poly = ['C**'+str(i) for i in range(10)]
        for c in c_poly:
            if c in eq: eq = eq.replace(c, 'C')
    return simplify_eq(eq)


def score_with_est_di(eq, tree_size, data_list, t_limit = 1.0, eta=0.999):
    """
    Calculate reward score for a complete parse tree 
    If placeholder C is in the equation, also excute estimation for C 
    Reward = 1 / (1 + MSE) * Penalty ** num_term 

    Parameters
    ----------
    eq : Str object.
        the discovered equation (with placeholders for coefficients). 
    tree_size : Int object.
        number of production rules in the complete parse tree. 
    data : 2-d numpy array.
        measurement data, including independent and dependent variables (last row). 
    t_limit : Float object.
        time limit (seconds) for single evaluation, default 1 second. 
        
    Returns
    -------
    score: Float
        discovered equations. 
    eq: Str
        discovered equations with estimated numerical values. 
    """
    rewards, eqs = [], []
    if isinstance(eq, list):
        initial_eqs = eq
    else:
        initial_eqs = [eq] * len(data_list)
    for env, data in enumerate(data_list):
        eq = initial_eqs[env]
        ## define independent variables and dependent variable
        num_var = data.shape[0] - 1
        if num_var <= 3: ## most cases ([x], [x,y], or [x,y,z])
            current_var = 'x'
            for i in range(num_var):
                globals()[current_var] = data[i, :]
                current_var = chr(ord(current_var) + 1)
            globals()['f_true'] = data[-1, :]
        else:            ## currently only double pendulum case has more than 3 independent variables
            globals()['x1'] = data[0, :]
            globals()['x2'] = data[1, :]
            globals()['w1'] = data[2, :]
            globals()['w2'] = data[3, :]
            globals()['wdot'] = data[4, :]
            globals()['f_true'] = data[5, :]


        ## count number of numerical values in eq
        c_count = eq.count('C')
        # start_time = time.time()
        with time_limit(t_limit, 'sleep'):
            try: 
                if c_count == 0:       ## no numerical values
                    f_pred = eval(eq)
                elif c_count >= 10:    ## discourage over complicated numerical estimations
                    return 0, eq
                else:                  ## with numerical values: coefficient estimationwith Powell method

                    # eq = prune_poly_c(eq)
                    c_lst = ['c'+str(i) for i in range(c_count)]
                    for c in c_lst: 
                        eq = eq.replace('C', c, 1)

                    def eq_test(c):
                        for i in range(len(c)): globals()['c'+str(i)] = c[i]
                        return np.linalg.norm(eval(eq) - f_true, 2)

                    x0 = [1.0] * len(c_lst)
                    c_lst = minimize(eq_test, x0, method='Powell', tol=1e-6).x.tolist() 
                    c_lst = [np.round(x, 4) if abs(x) > 1e-2 else 0 for x in c_lst]
                    eq_est = eq
                    for i in range(len(c_lst)):
                        eq_est = eq_est.replace('c'+str(i), str(c_lst[i]), 1)
                    eq = eq_est.replace('+-', '-')
                    f_pred = eval(eq)
            except: 
                return 0, eq

        r = float(eta ** tree_size / (1.0 + np.linalg.norm(f_pred - f_true, 2) ** 2 / f_true.shape[0]))
        rewards.append(r)
        eqs.append(eq)
    
    # run_time = np.round(time.time() - start_time, 3)
    # print('runtime :', run_time,  eq,  np.round(r, 3))
#     print(eqs[0] == eqs[1])
#     raise
    return np.mean(rewards), eqs



def run_spl(task, num_env,
            num_run, transplant_step, data_dir='data/', max_len = 50, eta = 0.9999, 
            max_module_init = 10, num_aug = 5, exp_rate = 1/np.sqrt(2), num_transplant = 20, 
            norm_threshold=1e-5, count_success = True):
    """
    Executes the main training loop of Symbolic Physics Learner.
    
    Parameters
    ----------
    task : String object.
        benchmark task name. 
    num_env : Int object.
        number of environments in the problem
    num_run : Int object.
        number of runs performed.
    transplant_step : Int object.
        number of iterations simulated for training between two transplantations. 
    data_dir : String object.
        directory of training data samples. 
    max_len : Int object.
        maximum allowed length (number of production rules ) of discovered equations.
    eta : Int object.
        penalty factor for rewarding. 
    max_module_init : Int object.
        initial maximum length for module transplantation candidates. 
    num_aug : Int object.
        number of trees for module transplantation. 
    exp_rate : Int object.
        initial exploration rate. 
    num_transplant : Int object.
        number of transplantation candidate update performed throughout traning. 
    norm_threshold : Float object.
        numerical error tolerance for norm calculation, a very small value. 
    count_success : Boolean object. 
        if success rate is recorded. 
        
    Returns
    -------
    all_eqs: List<Str>
        discovered equations. 
    success_rate: Float
        success rate of all runs performed. 
    all_times: List<Float>
        runtimes for successful runs. 
    """
    
    ## define production s and non-terminal nodes. 
    grammars = RULEMAP
    nt_nodes = ntn_map[task]


    ## read training and testing data
    train_samples, test_samples = [], []
    
    for env in range(num_env):
        train_sample = pd.read_csv(data_dir + task + f'_train_{env}.csv', header=None).to_numpy().T
        test_sample = pd.read_csv(data_dir + task + f'_test_{env}.csv', header=None).to_numpy().T
        train_samples.append(train_sample)
        test_samples.append(test_sample)
    
    num_success = 0
    all_times = []
    all_eqs = []
    
    ## number of module max size increase after each transplantation 
    module_grow_step = (max_len - max_module_init) / num_transplant

    for i_test in range(num_run):

        print("test", i_test)
        best_solution = ('nothing', 0)

        exploration_rate = exp_rate
        max_module = max_module_init
        reward_his = []
        best_modules = []
        aug_grammars = []

        start_time = time.time()
        discovery_time = 0

        for i_itr in range(num_transplant):

            spl_model = SplBase(data_sample = train_samples,
                                base_grammars = grammars, 
                                aug_grammars = aug_grammars, 
                                nt_nodes = nt_nodes, 
                                max_len = max_len, 
                                max_module = max_module,
                                aug_grammars_allowed = num_aug,
                                func_score = score_with_est_di, 
                                exploration_rate = exploration_rate, 
                                eta = eta)


            _, current_solution, good_modules = spl_model.run(transplant_step, 
                                                              num_play=10, 
                                                              print_flag=True)

            end_time = time.time() - start_time

            if not best_modules:
                best_modules = good_modules
            else:
                best_modules = sorted(list(set(best_modules + good_modules)), key = lambda x: x[1])
            aug_grammars = [x[0] for x in best_modules[-num_aug:]]
            
            # print([simplify_eq(x[2]) for x in best_modules[-num_aug:]])

            reward_his.append(best_solution[1])

            if current_solution[1] > best_solution[1]:
                best_solution = current_solution
            # print(best_solution)
            max_module += module_grow_step
            exploration_rate *= 5

#             print(best_solution)
            # check if solution is discovered. Early stop if it is. 
            test_score = score_with_est_di(simplify_eqs(best_solution[0]), 0, test_samples, eta = eta)[0]
            if test_score >= 1 - norm_threshold:
                num_success += 1
                if discovery_time == 0:
                    discovery_time = end_time
                    all_times.append(discovery_time)
                break

        
        all_eqs.append(simplify_eqs(best_solution[0]))
        print('\n{} tests complete after {} iterations.'.format(i_test+1, i_itr+1))
        print('best solution: {}'.format(simplify_eqs(best_solution[0])))
        print('test score: {}'.format(test_score))
        print()
    
    success_rate = num_success / num_run
    if count_success:
        print('success rate :', success_rate)
    
    return all_eqs, success_rate, all_times

In [10]:
output_folder = 'results_dsr/'  ## directory to save discovered results
save_eqs = True                 ## if true, discovered equations are saved to "output_folder" dir

In [12]:
for i in range(1,2):
    task = f'nguyen-{i}'
    print("="*30, task, "="*30)
#     run_spl(task, num_run, transplant_step, data_dir='data/', 
#             max_len = 50, eta = 0.9999, 
#             max_module_init = 10, num_aug = 5, exp_rate = 1/np.sqrt(2), 
#             num_transplant = 20, 
#             norm_threshold=1e-5, count_success = True):
    all_eqs, _, _ = run_spl(task, 
                            num_env=2,
                            num_run=1, 
                            max_len=50,
                            eta=1-1e-2, 
                            max_module_init=20,
                            num_transplant=20, 
                            num_aug=0,
                            transplant_step=1000, 
                            count_success=True)
# 0.976*x**3+-1.660*x**2+-1.280*x
# 4.304*x**3+4.406*x**2+-9.481*x


============================== nguyen-1 ==============================
test 0
Episode 1000/1000, current best reward 0.06208395211113493..

TypeError: unhashable type: 'list'